In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
import awoc

In [2]:
airport_raw = pd.read_csv('https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat', header = None)

In [3]:
airport_raw.columns = ['Airport ID', 'Name', 'City', 'Country', 'IATA', 'ICAO', 'Latitude',
                       'Longitude', 'Altitude', 'Timezone', 'DST', 'time zone', 'Type', 'Source']

In [4]:
airport_raw.head()

,Airport ID,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone,DST,time zone,Type,Source
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081690,145.391998,5282,10,U,Pacific/Port_Moresby,airport,OurAirports
1,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001,20,10,U,Pacific/Port_Moresby,airport,OurAirports
2,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,5388,10,U,Pacific/Port_Moresby,airport,OurAirports
3,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977,239,10,U,Pacific/Port_Moresby,airport,OurAirports
4,5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,147.220001,146,10,U,Pacific/Port_Moresby,airport,OurAirports


In [5]:
rout_raw = pd.read_csv('https://raw.githubusercontent.com/jpatokal/openflights/master/data/routes.dat', header = None)


In [6]:
rout_raw.head()

,0,1,2,3,4,5,6,7,8
0,2B,410,AER,2965,KZN,2990,NaN,0,CR2
1,2B,410,ASF,2966,KZN,2990,NaN,0,CR2
2,2B,410,ASF,2966,MRV,2962,NaN,0,CR2
3,2B,410,CEK,2968,KZN,2990,NaN,0,CR2
4,2B,410,CEK,2968,OVB,4078,NaN,0,CR2


In [7]:
rout_raw.columns = ['Airline', 'Airline ID', 'Source airport', 'Source airport ID', 'Destination airport',
 'Destination airport ID', 'Codeshare', 'Stops', 'Equipment']

In [8]:
rout_raw.head()

,Airline,Airline ID,Source airport,Source airport ID,Destination airport,Destination airport ID,Codeshare,Stops,Equipment
0,2B,410,AER,2965,KZN,2990,NaN,0,CR2
1,2B,410,ASF,2966,KZN,2990,NaN,0,CR2
2,2B,410,ASF,2966,MRV,2962,NaN,0,CR2
3,2B,410,CEK,2968,KZN,2990,NaN,0,CR2
4,2B,410,CEK,2968,OVB,4078,NaN,0,CR2


# Задание 1

In [9]:
rout_raw['Source airport'].value_counts()[:10]

ATL    915
ORD    558
PEK    535
LHR    527
CDG    524
FRA    497
LAX    492
DFW    469
JFK    456
AMS    453
Name: Source airport, dtype: int64

In [10]:
rout_raw['Destination airport'].value_counts()[:10]

ATL    911
ORD    550
PEK    534
LHR    524
CDG    517
LAX    498
FRA    493
DFW    467
JFK    455
AMS    450
Name: Destination airport, dtype: int64

In [11]:
source = pd.DataFrame({'Вылеты': rout_raw['Source airport'].value_counts()})
destination = pd.DataFrame({'Прибытия': rout_raw['Destination airport'].value_counts()})
new = source.join(destination)
new

,Вылеты,Прибытия
ATL,915,911.0
ORD,558,550.0
PEK,535,534.0
LHR,527,524.0
CDG,524,517.0
...,...,...
PPG,1,1.0
BLD,1,1.0
GCW,1,1.0
PRI,1,1.0


### Создаём одну табличку состаящую из кода IATA, вылетов и прилётов, уже отсортированную по убывнию.

In [12]:
new_n = new.reset_index()
new_n.rename(columns = {'index':'IATA'}, inplace = True )
new_n['Сумма'] = new_n['Вылеты'] + new_n['Прибытия']
new_n[:10]

,IATA,Вылеты,Прибытия,Сумма
0,ATL,915,911.0,1826.0
1,ORD,558,550.0,1108.0
2,PEK,535,534.0,1069.0
3,LHR,527,524.0,1051.0
4,CDG,524,517.0,1041.0
5,FRA,497,493.0,990.0
6,LAX,492,498.0,990.0
7,DFW,469,467.0,936.0
8,JFK,456,455.0,911.0
9,AMS,453,450.0,903.0


### Прихорашевам табличку и добавляем колонку с суммой 

In [13]:
# otvet = pd.merge(new_n, airport_raw)
otvet = new_n.merge(airport_raw, left_on = 'IATA', right_on = 'IATA')

### Объединяем 1-ую и нашу новую готовую таблички

In [14]:
otvet

,IATA,Вылеты,Прибытия,Сумма,Airport ID,Name,City,Country,ICAO,Latitude,Longitude,Altitude,Timezone,DST,time zone,Type,Source
0,ATL,915,911.0,1826.0,3682,Hartsfield Jackson Atlanta International Airport,Atlanta,United States,KATL,33.636700,-84.428101,1026,-5,A,America/New_York,airport,OurAirports
1,ORD,558,550.0,1108.0,3830,Chicago O'Hare International Airport,Chicago,United States,KORD,41.978600,-87.904800,672,-6,A,America/Chicago,airport,OurAirports
2,PEK,535,534.0,1069.0,3364,Beijing Capital International Airport,Beijing,China,ZBAA,40.080101,116.584999,116,8,U,Asia/Shanghai,airport,OurAirports
3,LHR,527,524.0,1051.0,507,London Heathrow Airport,London,United Kingdom,EGLL,51.470600,-0.461941,83,0,E,Europe/London,airport,OurAirports
4,CDG,524,517.0,1041.0,1382,Charles de Gaulle International Airport,Paris,France,LFPG,49.012798,2.550000,392,1,E,Europe/Paris,airport,OurAirports
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3247,MUK,1,1.0,2.0,5864,Mauke Airport,Mauke Island,Cook Islands,NCMK,-20.136101,-157.345001,26,-10,U,Pacific/Rarotonga,airport,OurAirports
3248,RUA,1,1.0,2.0,5709,Arua Airport,Arua,Uganda,HUAR,3.050000,30.917000,3951,3,U,Africa/Kampala,airport,OurAirports
3249,PPG,1,1.0,2.0,1970,Pago Pago International Airport,Pago Pago,American Samoa,NSTU,-14.331000,-170.710007,32,-11,U,Pacific/Pago_Pago,airport,OurAirports
3250,PRI,1,1.0,2.0,995,Praslin Airport,Praslin,Seychelles,FSPP,-4.319290,55.691399,10,4,U,Indian/Mahe,airport,OurAirports


In [15]:
otvet.drop(otvet.columns[[4,6,8,9,10,11,12,13,14,15,16]], axis= 1 , inplace= True )

### Удаляем ненужные колонки

In [16]:
otvet

,IATA,Вылеты,Прибытия,Сумма,Name,Country
0,ATL,915,911.0,1826.0,Hartsfield Jackson Atlanta International Airport,United States
1,ORD,558,550.0,1108.0,Chicago O'Hare International Airport,United States
2,PEK,535,534.0,1069.0,Beijing Capital International Airport,China
3,LHR,527,524.0,1051.0,London Heathrow Airport,United Kingdom
4,CDG,524,517.0,1041.0,Charles de Gaulle International Airport,France
...,...,...,...,...,...,...
3247,MUK,1,1.0,2.0,Mauke Airport,Cook Islands
3248,RUA,1,1.0,2.0,Arua Airport,Uganda
3249,PPG,1,1.0,2.0,Pago Pago International Airport,American Samoa
3250,PRI,1,1.0,2.0,Praslin Airport,Seychelles


In [17]:
otvet = otvet.set_index(['Country', 'Name'])

In [18]:
otvet.nlargest(10, 'Сумма')

IATA  Вылеты  \
Country        Name                                                            
United States  Hartsfield Jackson Atlanta International Airport  ATL     915   
               Chicago O'Hare International Airport              ORD     558   
China          Beijing Capital International Airport             PEK     535   
United Kingdom London Heathrow Airport                           LHR     527   
France         Charles de Gaulle International Airport           CDG     524   
Germany        Frankfurt am Main Airport                         FRA     497   
United States  Los Angeles International Airport                 LAX     492   
               Dallas Fort Worth International Airport           DFW     469   
               John F Kennedy International Airport              JFK     456   
Netherlands    Amsterdam Airport Schiphol                        AMS     453   

                                                                 Прибытия  \
Country        Name                                                         
United States  Hartsfield Jackson Atlanta International Airport     911.0   
               Chicago O'Hare International Airport                 550.0   
China          Beijing Capital International Airport                534.0   
United Kingdom London Heathrow Airport                              524.0   
France         Charles de Gaulle International Airport              517.0   
Germany        Frankfurt am Main Airport                            493.0   
United States  Los Angeles International Airport                    498.0   
               Dallas Fort Worth International Airport              467.0   
               John F Kennedy International Airport                 455.0   
Netherlands    Amsterdam Airport Schiphol                           450.0   

                                                                  Сумма  
Country        Name                                                      
United States  Hartsfield Jackson Atlanta International Airport  1826.0  
               Chicago O'Hare International Airport              1108.0  
China          Beijing Capital International Airport             1069.0  
United Kingdom London Heathrow Airport                           1051.0  
France         Charles de Gaulle International Airport           1041.0  
Germany        Frankfurt am Main Airport                          990.0  
United States  Los Angeles International Airport                  990.0  
               Dallas Fort Worth International Airport            936.0  
               John F Kennedy International Airport               911.0  
Netherlands    Amsterdam Airport Schiphol                         903.0

### Делаем колонки "Country" и "Name" индексами. Вот и всё, 1-ое задание готово!